In [1]:
import pandas as pd
import mysql.connector
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options as ChromeOptions

In [2]:
lista_productos = []  # Lista para almacenar los productos más baratos de todos los supermercados

categorias = ['Electro-y-tecnologia', 'Bazar-y-textil', 'Desayuno-y-merienda', 'Bebidas', 'Lacteos-y-productos-frescos', 
                'Carnes-y-Pescados', 'Frutas-y-Verduras', 'Panaderia', 'Congelados', 'Limpieza', 'Perfumeria', 'Mundo-Bebe', 'Mascotas']

for categoria in categorias:
    encontrado = False
    x = 1  # Variable para controlar las páginas

    while not encontrado:
        try:
            lista_productos_categoria = []  # Lista para almacenar los productos más baratos del supermercado actual

            # Configurar el driver de Selenium y cargar la página web
            options = ChromeOptions()
            options.add_argument("--start-minimized")
            driver = webdriver.Chrome(options=options)
            
            supermercado = 'https://www.carrefour.com.ar/'
    
            driver.get(f'{supermercado}{categoria}{"?map=category-1&page="}{x}')

            super = supermercado.split('.')[1].upper()

            # Esperar hasta que el elemento sea visible
            wait = WebDriverWait(driver, 7)
            
            try:
                elemento = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span.valtech-carrefourar-product-price-0-x-currencyContainer')))
            except:
                continue

            # Obtener el contenido de la página web con BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Obtenemos todos los elementos contenedores de productos
            productos = soup.find_all('div', 
                                        {'class': 'valtech-carrefourar-product-summary-status-0-x-container valtech-carrefourar-product-summary-status-0-x-productNotAdded flex flex-column h-100'})

            for producto in productos:
                try:
                    # Precio con Tarjeta Carrefour
                    precio = producto.find('span', 
                                        {'class': 'valtech-carrefourar-product-price-0-x-currencyContainer'}).text

                    # Precio sin Tarjeta Carrefour
                    #precio = producto.find('span', 
                                        #{'class': 'valtech-carrefourar-product-price-0-x-listPriceValue strike'}).text
                except AttributeError:
                    continue
                
                # Guardamos el nombre de los productos
                nombre = producto.find('h3', 
                                        {'class': 'vtex-product-summary-2-x-productNameContainer mv0 vtex-product-summary-2-x-nameWrapper overflow-hidden c-on-base f5'}).span.text
                
                try:
                    # Promo Carrefour %OFF
                    promo = producto.find('span', 
                                            {'class': 'valtech-carrefourar-product-price-0-x-discountPercentage f6 lh-copy'}).text.strip()
                except AttributeError:
                    try:
                        # Promo Carrefour con tarjeta Carrefour
                        promo = producto.find('span', 
                                                {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).text
                    
                        # A revisar, agregar una nueva columna de precio sin promocion, en caso de que se necesite usar tarjeta para la misma.
                    
                    except AttributeError:
                        try:
                            # Promo Carrefour con tarjeta Carrefour
                            promo = producto.find('span', 
                                                    {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).text
                            
                            # Promo Carrefour Adicional
                            promo2 = producto.find('span', 
                                                    {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).span.text

                            promo += promo2
                            
                        except AttributeError:
                            try:
                                # Promo Carrefour Adicional
                                promo = producto.find('span', 
                                                        {'class': 'valtech-carrefourar-product-highlights-0-x-productHighlightText'}).span.text
                            except AttributeError:
                                    promo = 'No tiene promo'
                                    
                if 'MI CRF' in promo:
                    precio_sin_promo = producto.find('span', 
                                                {'class': 'valtech-carrefourar-product-price-0-x-listPriceValue strike'}).text
                else:
                    precio_sin_promo = precio
                
                url = producto.find('a', 
                                    {'class': 'vtex-product-summary-2-x-clearLink'})['href']

                url.split('/')[1]

                precios = [precio, precio_sin_promo]

                for precios_int in precios:
                    precios_int = precios_int.replace('.', '')
                    precios_int = precios_int.replace(',', '.')
                    precios_int = precios_int.replace('$', '')
                    precios_int = float(precios_int)

                producto_dict = {
                    'supermercado': super,
                    'nombre': nombre,
                    'precio': precio,
                    'promo': promo,
                    'url': supermercado + url,
                    'precio sin promo': precio_sin_promo
                }

                lista_productos_categoria.append(producto_dict)

            lista_productos.extend(lista_productos_categoria)

            x += 1

        except:
            elemento = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located(
                    (By.CSS_SELECTOR, 'div.valtech-carrefourar-search-result-0-x-searchNotFound')))
            encontrado = True

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.201)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008DA813+48355]
	(No symbol) [0x0086C4B1]
	(No symbol) [0x00775358]
	(No symbol) [0x0075D293]
	(No symbol) [0x007BE37B]
	(No symbol) [0x007CC473]
	(No symbol) [0x007BA536]
	(No symbol) [0x007982DC]
	(No symbol) [0x007993DD]
	GetHandleVerifier [0x00B3AABD+2539405]
	GetHandleVerifier [0x00B7A78F+2800735]
	GetHandleVerifier [0x00B7456C+2775612]
	GetHandleVerifier [0x009651E0+616112]
	(No symbol) [0x00875F8C]
	(No symbol) [0x00872328]
	(No symbol) [0x0087240B]
	(No symbol) [0x00864FF7]
	BaseThreadInitThunk [0x762E7D59+25]
	RtlInitializeExceptionChain [0x774FB79B+107]
	RtlClearBits [0x774FB71F+191]
